In [34]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras.models import Model

In [35]:
# Define your data directories, batch size, and other hyperparameters
train_dir = 'final_dataset/train'
val_dir = 'final_dataset/validation'
test_dir = 'final_dataset/test'
# Define input shape and number of classes
input_shape = (299, 299, 3)  # Adjust input shape based on the requirements
num_classes = 101
num_epochs = 100

In [36]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 75750 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
Found 15150 images belonging to 101 classes.


In [37]:
# Define Callbacks
checkpoint = ModelCheckpoint("customModel1.h5", save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, mode='min', verbose=1)

In [38]:
# Define the CNN model
def customModel1(input_shape, num_classes):
    model = models.Sequential()

    # Add a Convolutional Layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Add another Convolutional Layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
              
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output before the fully connected layers
    model.add(layers.Flatten())
    
    # Fully Connected Layers
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [39]:
model = customModel1(input_shape, num_classes)

In [40]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 297, 297, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 148, 148, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 35, 35, 128)     

In [42]:
import time
start_time = time.time()

In [43]:
# Train the model
history = model.fit(
        train_generator,
        epochs=num_epochs,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/100
2368/2368 [==============================] - ETA: 0s - loss: 4.3958 - accuracy: 0.0436
Epoch 1: val_loss improved from inf to 4.05184, saving model to customModel1.h5
2368/2368 [==============================] - 1173s 495ms/step - loss: 4.3958 - accuracy: 0.0436 - val_loss: 4.0518 - val_accuracy: 0.0916 - lr: 1.0000e-04
Epoch 2/100
2368/2368 [==============================] - ETA: 0s - loss: 4.1112 - accuracy: 0.0813
Epoch 2: val_loss improved from 4.05184 to 3.83988, saving model to customModel1.h5
2368/2368 [==============================] - 1172s 495ms/step - loss: 4.1112 - accuracy: 0.0813 - val_loss: 3.8399 - val_accuracy: 0.1316 - lr: 1.0000e-04
Epoch 3/100
2368/2368 [==============================] - ETA: 0s - loss: 3.9250 - accuracy: 0.1097
Epoch 3: val_loss improved from 3.83988 to 3.63893, saving model to customModel1.h5
2368/2368 [==============================] - 1172s 495ms/step - loss: 3.9250 - accuracy: 0.1097 - val_loss: 3.6389 - val_accuracy: 0.1667 - lr: 1

Epoch 26/100
2368/2368 [==============================] - ETA: 0s - loss: 2.8934 - accuracy: 0.2968
Epoch 26: val_loss improved from 2.66347 to 2.66275, saving model to customModel1.h5
2368/2368 [==============================] - 1172s 495ms/step - loss: 2.8934 - accuracy: 0.2968 - val_loss: 2.6627 - val_accuracy: 0.3509 - lr: 1.0000e-04
Epoch 27/100
2368/2368 [==============================] - ETA: 0s - loss: 2.8819 - accuracy: 0.2989
Epoch 27: val_loss improved from 2.66275 to 2.60585, saving model to customModel1.h5
2368/2368 [==============================] - 1173s 496ms/step - loss: 2.8819 - accuracy: 0.2989 - val_loss: 2.6058 - val_accuracy: 0.3610 - lr: 1.0000e-04
Epoch 28/100
2368/2368 [==============================] - ETA: 0s - loss: 2.8618 - accuracy: 0.3036
Epoch 28: val_loss did not improve from 2.60585
2368/2368 [==============================] - 1163s 491ms/step - loss: 2.8618 - accuracy: 0.3036 - val_loss: 2.6291 - val_accuracy: 0.3585 - lr: 1.0000e-04
Epoch 29/100
2368

2368/2368 [==============================] - ETA: 0s - loss: 2.5479 - accuracy: 0.3694
Epoch 51: val_loss did not improve from 2.38081
2368/2368 [==============================] - 1166s 493ms/step - loss: 2.5479 - accuracy: 0.3694 - val_loss: 2.3854 - val_accuracy: 0.4105 - lr: 1.0000e-05
Epoch 52/100
2368/2368 [==============================] - ETA: 0s - loss: 2.5459 - accuracy: 0.3686
Epoch 52: val_loss improved from 2.38081 to 2.36418, saving model to customModel1.h5
2368/2368 [==============================] - 1172s 495ms/step - loss: 2.5459 - accuracy: 0.3686 - val_loss: 2.3642 - val_accuracy: 0.4140 - lr: 1.0000e-05
Epoch 53/100
2368/2368 [==============================] - ETA: 0s - loss: 2.5487 - accuracy: 0.3684
Epoch 53: val_loss did not improve from 2.36418
2368/2368 [==============================] - 1172s 495ms/step - loss: 2.5487 - accuracy: 0.3684 - val_loss: 2.3799 - val_accuracy: 0.4136 - lr: 1.0000e-05
Epoch 54/100
2368/2368 [==============================] - ETA: 0s -

In [44]:
end_time = time.time()
training_time = end_time - start_time
print(f"Total training time: {training_time:.2f} seconds")

Total training time: 66628.80 seconds


In [45]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2%}")

474/474 [==============================] - 37s 77ms/step - loss: 2.3755 - accuracy: 0.4150
Test Loss: 2.3755
Test Accuracy: 41.50%
